In [1]:
import sys
sys.path.append("..")
import logging
import random
import neptune
import numpy as np
import syft as sy
from torch import load
from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
from federated_learning.FederatedLearning import FederatedLearning
from federated_learning.helper import utils
# arguments = docopt(__doc__)
CONFIG_PATH = '../configs/defaults.yml'

arguments = dict()
arguments['--reg'] = 0.0
arguments['--output-prefix'] = "mnist_no_attack"
arguments['--server_model'] = "data_output/20201107_030649_mnist_w0/server_model_9"
arguments["--epoch"] = 1
arguments["--round"] = 1
output_dir = None
log_enable = False
neptune_enable = False


configs = utils.load_config(CONFIG_PATH)
logging.basicConfig(format='%(asctime)s %(message)s', level=configs['log']['level'])
random.seed(configs['runtime']['random_seed'])

# From command line
epochs_num = int(arguments["--epoch"])
rounds_num = int(arguments["--round"])

train_workers_num = configs['runtime']['train_workers_num']

fl = FederatedLearning(
    configs['runtime']['batch_size'], 
    configs['runtime']['test_batch_size'], 
    configs['runtime']['lr'], 
    float(arguments['--reg']) if arguments['--reg'] is not None else 0.0,
    configs['runtime']['momentum'], 
    neptune_enable, log_enable, 
    configs['log']['interval'], 
    output_dir, 
    configs['runtime']['random_seed'])

fl.create_server()
fl.create_server_model()
workers_idx = ["worker_" + str(i) for i in range(configs['runtime']['mnist_workers_num'])]
fl.create_workers(workers_idx)
fl.create_workers_model(workers_idx)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ubuntu/.local/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


2020-11-08 22:49:37,879 Initializing Federated Learning class...
2020-11-08 22:49:38,642 Creating the server...
2020-11-08 22:49:38,643 Creating a model for the server...
2020-11-08 22:49:38,653 Creating workers...
2020-11-08 22:49:38,656 Creating a model for 30 worker(s)...


In [2]:
train_raw_dataset = utils.preprocess_mnist(
    utils.load_mnist_data_train(
        configs['data']['MNIST_PATH'], 
        configs['runtime']['mnist_data_percentage']))
train_dataset = utils.get_mnist_dataset(train_raw_dataset)
train_dataloader = utils.get_mnist_dataloader(
    train_dataset, configs['runtime']['batch_size'], shuffle=True)

test_data = utils.load_mnist_data_test(configs['data']['MNIST_PATH'])
test_dataset = utils.get_mnist_dataset(test_data)
test_dataloader = utils.get_mnist_dataloader(
    test_dataset, configs['runtime']['test_batch_size'], shuffle=True)

server_dataset = utils.get_server_dataset(
    train_dataloader, configs['runtime']['public_data_percentage'])
federated_server_dataloader = fl.create_federated_mnist(
    server_dataset, ["server"], configs['runtime']['batch_size'], shuffle=False)

2020-11-08 22:49:41,566 Loading 100% of train data from MNIST dataset.
2020-11-08 22:49:41,830 Preparing the MNIST dataset.
2020-11-08 22:49:42,011 Creating MNIST dataset.
2020-11-08 22:49:42,012 Creating MNIST data loader.
2020-11-08 22:49:42,014 Loading test data from MNIST dataset.
2020-11-08 22:49:42,039 Creating MNIST dataset.
2020-11-08 22:49:42,041 Creating MNIST data loader.
2020-11-08 22:49:42,042 Creating server MNIST data loader.
2020-11-08 22:49:42,057 0.00% Loaded...
2020-11-08 22:49:42,641 8.33% Loaded...
2020-11-08 22:49:43,279 16.67% Loaded...
2020-11-08 22:49:44,378 25.00% Loaded...
2020-11-08 22:49:46,003 33.33% Loaded...
2020-11-08 22:49:47,970 41.67% Loaded...
2020-11-08 22:49:50,220 50.00% Loaded...
2020-11-08 22:49:52,928 58.33% Loaded...
2020-11-08 22:49:56,152 66.67% Loaded...
2020-11-08 22:49:59,819 75.00% Loaded...
2020-11-08 22:50:03,859 83.33% Loaded...
2020-11-08 22:50:08,322 91.67% Loaded...
2020-11-08 22:50:13,195 Scanning and sending data to server...


In [ ]:
len(train_raw_dataset['x'])

In [ ]:
len(train_raw_dataset['x'][:int((30/100)* len(train_raw_dataset['x']))])

In [ ]:
for id_, (data, target) in enumerate(test_dataloader):
    print(id_, data.shape, target.shape)
    break

In [ ]:
dir(test_dataloader)

In [ ]:
id, (data, target) = next(enumerate(test_dataloader))

In [ ]:
batch_idx, (data, target) = next(enumerate(federated_server_dataloader))

In [ ]:
target.get()

In [ ]:
from datetime import datetime
import time
time.strftime("%Y%m%d_%H%M%S")

In [ ]:
import os

In [ ]:
os.mkdir("../data_output/ehsan1/")